In [2]:
import numpy as np

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import random
import sys
import glob
sys.path.append("./")

2023-06-25 13:37:25.097094: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-25 13:37:25.155972: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#### Configuration file

In [5]:
from signet.configs.Conv1D_LSTM_CTC_Loss import Conv1D_LSTM_CTC_Loss
CFG = Conv1D_LSTM_CTC_Loss()

#### Set random seeds and training strategy

In [6]:
# Seed all random number generators
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
def get_strategy(CFG):
    if CFG.device == "GPU"  or CFG.device=="CPU":
        ngpu = len(tf.config.experimental.list_physical_devices('GPU'))
        if ngpu>1:
            print("Using multi GPU")
            strategy = tf.distribute.MirroredStrategy()
        elif ngpu==1:
            print("Using single GPU")
            strategy = tf.distribute.get_strategy()
        else:
            print("Using CPU")
            strategy = tf.distribute.get_strategy()
            CFG.device = "CPU"

    AUTO     = tf.data.experimental.AUTOTUNE
    REPLICAS = strategy.num_replicas_in_sync
    print(f'REPLICAS: {REPLICAS}')
    
    return strategy, REPLICAS

STRATEGY, N_REPLICAS = get_strategy(CFG)

Using single GPU
REPLICAS: 1


#### Prepare dataset for training 

In [7]:
from signet.dataset.utils import get_tfrec_dataset

In [8]:
ALL_FILENAMES = glob.glob('../dataset/tdf_data/*.tfrecords')
print(len(ALL_FILENAMES))

120165


In [9]:
ROOT_DIR = "../dataset"
supp_df = pd.read_csv("../dataset/supplemental_metadata.csv")
train_df = pd.read_csv("../dataset/train.csv")

In [10]:
ds = get_tfrec_dataset(ALL_FILENAMES,CFG)
for x in ds:
    temp_train = x
    break
inp,y_true = temp_train

2023-06-25 13:37:32.458578: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38121 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:3d:00.0, compute capability: 8.0
2023-06-25 13:37:33.928220: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [120165]
	 [[{{node Placeholder/_0}}]]
2023-06-25 13:37:33.928699: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [120165]
	 [[{{node Placeholder/_0}}]]


#### Prepare model

In [11]:
# from signet.trainer.ctc_loss_trainer import train_model

In [12]:
import tensorflow as tf
from signet.models.feature_extractor import Cnn1dMhsaFeatureExtractor
from signet.configs.Conv1D_LSTM_CTC_Loss import Conv1D_LSTM_CTC_Loss
import numpy as np
CFG = Conv1D_LSTM_CTC_Loss()

In [13]:
model = Cnn1dMhsaFeatureExtractor(CFG)
y_pred = model(inp)

2023-06-25 13:37:37.289077: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-25 13:37:37.392476: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8901


In [18]:
def ctc_decode(y_pred,blank_index=59,merge_repeated=True):
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int32")
    batch_len = tf.cast(tf.shape(y_pred)[0], dtype="int32")

    inplen = input_length * tf.ones(shape=(batch_len,), dtype="int32")
    outs = tf.nn.ctc_greedy_decoder(tf.transpose(y_pred,(1,0,2)),inplen,blank_index=blank_index,merge_repeated=merge_repeated)
    decoded = tf.sparse.to_dense(outs[0][0],default_value=-1)
    for true_seq,pred_seq in zip(y_true.numpy(),decoded.numpy()):
        true_seq = true_seq[true_seq!=-1]
        pred_seq = pred_seq[pred_seq!=-1]
        break

(array([52, 21, 52, 21, 52, 21, 52, 21, 52, 21, 52, 55, 52, 21, 52, 55, 52,
        21, 52, 21, 52, 21, 52, 21, 52, 55, 52, 55, 52, 55, 21, 52, 21, 52,
        21, 52, 21, 52, 21, 52, 21, 55, 21, 52, 21, 52, 55, 52, 21, 55, 21,
        52, 55, 52, 21,  0, 21, 52, 25, 55,  0, 55, 25, 55, 52,  0, 52, 21,
        52, 21, 52, 21, 52, 21, 52, 21, 55,  0, 21, 52, 21, 55,  0, 55, 21,
        25, 21]),
 array([44, 40, 34, 49, 46, 50, 34, 46, 47, 36, 50,  0, 44, 32, 42, 36,  0,
        50, 44, 32, 43, 43,  0, 51, 39, 40, 45, 38, 50,  0, 43, 46, 46, 42,
         0, 33, 40, 38]))

In [28]:
ctc_decode(y_pred).numpy().shape,y_true.shape

((64, 88), TensorShape([64, 45]))

In [30]:
loss = tf.nn.ctc_loss(label,answer,lenout,leninp,blank_index=59,logits_time_major=False)

Instructions for updating:
Prefer tf.tensor_scatter_nd_add, which offers the same functionality with well-defined read-write semantics.
Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.
